In [1]:
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os

from astropy import wcs
from astropy.io import fits
import sys

# from astroquery.simbad import Simbad
# from astroquery.vizier import Vizier

from astropy.table import Table, vstack
from astropy.table import Column

import collections
import subprocess
from astropy.io import ascii
import time
from astropy.utils.data import clear_download_cache
from astroquery.gaia import Gaia
# import warnings
# warnings.filterwarnings('ignore')
import pandas as pd

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [42]:
def findMatchesNoCondition(tab,match,max_sep = 5 * u.arcsec):
    
    df = tab.to_pandas()
    bigdf = match.to_pandas()
    
    number1 = match['index']
    ra1 = match['RA']
    dec1 = match['DEC']

    number2 = tab['Name'] # difference image
    ra2 = tab['RA(deg)']
    dec2 = tab['Dec(deg)']    
    
    cat1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
    cat2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
    
    idx, d2d, d3d = cat2.match_to_catalog_sky(cat1)
    sep_constraint = d2d < max_sep
    objectsFound = number2[sep_constraint]
    matchdf = df[df['Name'].isin(objectsFound)]
    matchTab = Table.from_pandas(matchdf)

    idx, d2d, d3d = cat1.match_to_catalog_sky(cat2)
    sep_constraint = d2d < max_sep
    #istrue = [x for x in sep_constraint if x == True]
    objectsFound = number1[sep_constraint]
    origdf = bigdf[bigdf['index'].isin(objectsFound)]
    origTab = Table.from_pandas(origdf)
    
    return [matchTab,origTab]

In [43]:
tabs = findMatchesNoCondition(cvTab,kepler)

In [44]:
for x in tabs:
    ascii
    print(len(x))

4
6


In [41]:
ascii.write(tabs[0],'/home/mj1e16/outTablesImproved/cvcv25.csv',format='csv',fast_writer=False)
ascii.write(tabs[1],'/home/mj1e16/outTablesImproved/cvkep25.csv',format='csv',fast_writer=False)

In [3]:
cvTab = ascii.read('/home/mj1e16/Downloads/CVs_masterlist_20180424.csv')

In [31]:
kepler = ascii.read('/home/mj1e16/outTables15/firstPassTransients15.csv')

In [33]:
indexes = range(len(kepler))
kepler['index'] = indexes

In [46]:
len(cvTab)

8048